In [34]:
%matplotlib inline
import imp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
import seaborn as sns; sns.set()
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import BaseEstimator
from sklearn.model_selection import GridSearchCV
from time import time
from sklearn.metrics import mean_squared_error

In [123]:
#Join iata data
iata = pd.read_csv('data/IATA_US_CODES.csv')
iata['IATA Code']=iata['IATA Code'].astype(str)
iata['IATA Code']=iata['IATA Code'].str.rstrip()
iata_dep=iata.add_suffix('_Dep')
iata_arr=iata.add_suffix('_Arr')
iata['2018']=iata['2018'].str.replace(',','')
iata['2017']=iata['2017'].str.replace(',','')
iata['2016']=iata['2016'].str.replace(',','')
iata['2015']=iata['2015'].str.replace(',','')
iata['2018']=iata['2018'].str.replace('\xa0','')
iata['2017']=iata['2017'].str.replace('\xa0','')
iata['2016']=iata['2016'].str.replace('\xa0','')
iata['2015']=iata['2015'].str.replace('\xa0','')
iata['State']=iata['State'].str.replace('\xa0','')
iata['Major city served']=iata['Major city served'].str.replace('New York/','')
iata['Major city served']=iata['Major city served'].str.replace('/St. Paul','')
iata['Major city served']=iata['Major city served'].str.replace('Baltimore/','')
iata['Major city served']=iata['Major city served'].str.replace('/Tacoma','')
iata['Major city served']=iata['Major city served'].str.replace('/Fort Worth','')
iata['Major city served']=iata['Major city served'].str.replace(', D.C.','')
iata

,Rank_2018,Airports (large hubs),IATA Code,Major city served,State,2018,2017,2016,2015
0,1,Hartsfield–Jackson Atlanta International Airport,ATL,Atlanta,GA,51866464,50251964,50501858,49340732
1,2,Los Angeles International Airport,LAX,Los Angeles,CA,42626783,41232432,39636042,36351226
2,3,O'Hare International Airport,ORD,Chicago,IL,39874879,38593028,37589899,36305668
3,4,Dallas/Fort Worth International Airport,DFW,Dallas,TX,32800721,31816933,31283579,31589832
4,5,Denver International Airport,DEN,Denver,CO,31363573,29809097,28267394,26280043
5,6,John F. Kennedy International Airport,JFK,New York,NY,30541459,29533154,29239151,27782369
6,7,San Francisco International Airport,SFO,San Francisco,CA,27794154,26900048,25707101,24190549
7,8,Seattle–Tacoma International Airport,SEA,Seattle,WA,24894338,22639124,21887110,21231781
8,9,McCarran International Airport,LAS,Las Vegas,NV,23655285,23364393,22833267,21824231
9,10,Orlando International Airport,MCO,Orlando,FL,23184634,21565448,20283541,18759938


In [70]:
data_weather = pd.read_csv('submissions/starting_kit_2/external_data.csv')
data_weather=data_weather.join(iata, on='AirPort', how='left')
data_weather['Major city served']=data_weather['Major city served'].str.replace('\xa0','')

data_weather['Major city served'].unique()

array(['Atlanta', 'Chicago', 'Los Angeles', 'Dallas', 'Denver',
       'New York', 'San Francisco', 'Charlotte', 'Las Vegas', 'Phoenix',
       'Houston', 'Miami', 'Orlando', 'Newark', 'Seattle', 'Minneapolis',
       'Detroit', 'Philadelphia', 'Boston'], dtype=object)

In [71]:
us_cities = pd.read_csv('data/uscities.csv')

In [72]:
data_weather=data_weather.merge(us_cities, left_on=['Major city served', 'State'],right_on=['city', 'state_id'], how='left')

In [73]:
data_weather

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Rank_2018,Airports (large hubs),Major city served,State,2018,2017,2016,2015,city,city_ascii,state_id,state_name,county_fips,county_name,county_fips_all,county_name_all,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,1,Hartsfield–Jackson Atlanta International Airport,Atlanta,GA,51866464,50251964,50501858,49340732,Atlanta,Atlanta,GA,Georgia,13121,Fulton,13121|13089|13067,Fulton|DeKalb|Cobb,33.7627,-84.4225,5228750.0,1384.0,polygon,False,True,America/New_York,1,30334 30331 30332 30309 30308 30305 30307 3030...,1840016319
1,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,NaN,185,1,Hartsfield–Jackson Atlanta International Airport,Atlanta,GA,51866464,50251964,50501858,49340732,Atlanta,Atlanta,GA,Georgia,13121,Fulton,13121|13089|13067,Fulton|DeKalb|Cobb,33.7627,-84.4225,5228750.0,1384.0,polygon,False,True,America/New_York,1,30334 30331 30332 30309 30308 30305 30307 3030...,1840016319
2,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147,1,Hartsfield–Jackson Atlanta International Airport,Atlanta,GA,51866464,50251964,50501858,49340732,Atlanta,Atlanta,GA,Georgia,13121,Fulton,13121|13089|13067,Fulton|DeKalb|Cobb,33.7627,-84.4225,5228750.0,1384.0,polygon,False,True,America/New_York,1,30334 30331 30332 30309 30308 30305 30307 3030...,1840016319
3,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139,1,Hartsfield–Jackson Atlanta International Airport,Atlanta,GA,51866464,50251964,50501858,49340732,Atlanta,Atlanta,GA,Georgia,13121,Fulton,13121|13089|13067,Fulton|DeKalb|Cobb,33.7627,-84.4225,5228750.0,1384.0,polygon,False,True,America/New_York,1,30334 30331 30332 30309 30308 30305 30307 3030...,1840016319
4,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149,1,Hartsfield–Jackson Atlanta International Airport,Atlanta,GA,51866464,50251964,50501858,49340732,Atlanta,Atlanta,GA,Georgia,13121,Fulton,13121|13089|13067,Fulton|DeKalb|Cobb,33.7627,-84.4225,5228750.0,1384.0,polygon,False,True,America/New_York,1,30334 30331 30332 30309 30308 30305 30307 3030...,1840016319
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11035,2013-03-01,LGA,7,5,3,-1,-3,-4,76,63,49,1008,1005,1002,16,16,16,34,21,42.0,0.00,6,NaN,320,21,LaGuardia Airport,New York,NY,15059648,14614802,14706123,14762593,New York,New York,NY,New York,36061,New York,36061,New York,40.6943,-73.9249,19354922.0,11083.0,polygon,False,True,America/New_York,1,11229 11226 11225 11224 11222 11221 11220 1138...,1840059961
11036,2013-03-02,LGA,4,2,0,-2,-5,-6,82,65,48,1008,1007,1006,16,15,6,34,20,40.0,T,8,Snow,317,21,LaGuardia Airport,New York,NY,15059648,14614802,14706123,14762593,New York,New York,NY,New York,36061,New York,36061,New York,40.6943,-73.9249,19354922.0,11083.0,polygon,False,True,America/New_York,1,11229 11226 11225 11224 11222 11221 11220 1138...,1840059961
11037,2013-03-03,LGA,4,2,-1,-5,-8,-9,69,55,40,1008,1006,1004,16,15,8,39,24,50.0,T,6,Snow,314,21,LaGuardia Airport,New York,NY,15059648,14614802,14706123,14762593,New York,New York,NY,New York,36061,New York,36061,New York,40.6943,-73.9249,19354922.0,11083.0,polygon,False,True,America/New_York,1,11229 11226 11225 11224 11222 11221 11220 1138...,1840059961
11038,2013-03-04,LGA,5,2,-2,-7,-8,-9,63,54,44,1012,1009,1008,16,16,1

In [74]:
data_weather=data_weather.drop(['city_ascii','state_id','state_name','county_fips','county_name',
                                'county_fips_all','county_name_all','source','military',
                                'incorporated','timezone','zips','id','Major city served',
                               'Airports (large hubs)'], axis=1)

In [75]:
data_weather

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Rank_2018,State,2018,2017,2016,2015,city,lat,lng,population,density,ranking
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1
1,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,NaN,185,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1
2,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1
3,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1
4,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11035,2013-03-01,LGA,7,5,3,-1,-3,-4,76,63,49,1008,1005,1002,16,16,16,34,21,42.0,0.00,6,NaN,320,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1
11036,2013-03-02,LGA,4,2,0,-2,-5,-6,82,65,48,1008,1007,1006,16,15,6,34,20,40.0,T,8,Snow,317,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1
11037,2013-03-03,LGA,4,2,-1,-5,-8,-9,69,55,40,1008,1006,1004,16,15,8,39,24,50.0,T,6,Snow,314,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1
11038,2013-03-04,LGA,5,2,-2,-7,-8,-9,63,54,44,1012,1009,1008,16,16,16,47,31,60.0,0.00,3,NaN,313,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1


In [76]:
from math import *

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [77]:
fuel = pd.read_csv('data/fuelprices.csv')
fuel=fuel.drop('Unnamed: 0',axis=1)
list(fuel['Date'].unique())

['2011-09-01',
 '2011-09-02',
 '2011-09-06',
 '2011-09-07',
 '2011-09-08',
 '2011-09-09',
 '2011-09-12',
 '2011-09-13',
 '2011-09-14',
 '2011-09-15',
 '2011-09-16',
 '2011-09-19',
 '2011-09-20',
 '2011-09-21',
 '2011-09-22',
 '2011-09-23',
 '2011-09-26',
 '2011-09-27',
 '2011-09-28',
 '2011-09-29',
 '2011-09-30',
 '2011-10-03',
 '2011-10-04',
 '2011-10-05',
 '2011-10-06',
 '2011-10-07',
 '2011-10-10',
 '2011-10-11',
 '2011-10-12',
 '2011-10-13',
 '2011-10-14',
 '2011-10-17',
 '2011-10-18',
 '2011-10-19',
 '2011-10-20',
 '2011-10-21',
 '2011-10-24',
 '2011-10-25',
 '2011-10-26',
 '2011-10-27',
 '2011-10-28',
 '2011-10-31',
 '2011-11-01',
 '2011-11-02',
 '2011-11-03',
 '2011-11-04',
 '2011-11-07',
 '2011-11-08',
 '2011-11-09',
 '2011-11-10',
 '2011-11-11',
 '2011-11-14',
 '2011-11-15',
 '2011-11-16',
 '2011-11-17',
 '2011-11-18',
 '2011-11-21',
 '2011-11-22',
 '2011-11-23',
 '2011-11-25',
 '2011-11-28',
 '2011-11-29',
 '2011-11-30',
 '2011-12-01',
 '2011-12-02',
 '2011-12-05',
 '2011-12-

In [78]:
data_weather=data_weather.merge(fuel, left_on='Date', right_on='Date', how='left')

In [79]:
data_weather[data_weather['U.S. Gulf Coast Kerosene-Type Jet Fuel Spot Price FOB (Dollars per Gallon)'].isna()]

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Rank_2018,State,2018,2017,2016,2015,city,lat,lng,population,density,ranking,U.S. Gulf Coast Kerosene-Type Jet Fuel Spot Price FOB (Dollars per Gallon)
2,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,NaN
3,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,NaN
4,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,NaN
9,2011-09-10,ATL,29,22,14,13,11,7,84,56,28,1019,1016,1015,16,16,16,21,9,32.0,0.00,0,NaN,321,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,NaN
10,2011-09-11,ATL,30,23,16,13,11,7,72,49,25,1020,1018,1017,16,16,16,21,6,37.0,0.00,2,NaN,294,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11024,2013-02-18,LGA,2,-2,-7,-12,-15,-17,51,39,26,1024,1022,1016,16,16,16,50,26,64.0,0.00,1,NaN,278,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,NaN
11029,2013-02-23,LGA,4,3,2,2,1,-1,92,84,76,1025,1017,1009,16,7,0,34,18,42.0,6.86,8,Rain,53,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,NaN
11030,2013-02-24,LGA,8,5,2,2,-1,-6,92,73,53,1014,1009,1007,16,14,5,45,22,56.0,0.76,7,Rain-Snow,327,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,NaN
11036,2013-03-02,LGA,4,2,0,-2,-5,-6,82,65,48,1008,1007,1006,16,15,6,34,20,40.0,T,8,Snow,317,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,NaN


In [80]:
data_weather = data_weather.rename(
        columns={'U.S. Gulf Coast Kerosene-Type Jet Fuel Spot Price FOB (Dollars per Gallon)': 'Fuel_price'})

In [81]:
data_weather

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Rank_2018,State,2018,2017,2016,2015,city,lat,lng,population,density,ranking,Fuel_price
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,"3,088"
1,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,NaN,185,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,"3,038"
2,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,NaN
3,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,NaN
4,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11035,2013-03-01,LGA,7,5,3,-1,-3,-4,76,63,49,1008,1005,1002,16,16,16,34,21,42.0,0.00,6,NaN,320,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,"3,016"
11036,2013-03-02,LGA,4,2,0,-2,-5,-6,82,65,48,1008,1007,1006,16,15,6,34,20,40.0,T,8,Snow,317,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,NaN
11037,2013-03-03,LGA,4,2,-1,-5,-8,-9,69,55,40,1008,1006,1004,16,15,8,39,24,50.0,T,6,Snow,314,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,NaN
11038,2013-03-04,LGA,5,2,-2,-7,-8,-9,63,54,44,1012,1009,1008,16,16,16,47,31,60.0,0.00,3,NaN,313,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,"3,005"


In [82]:
data_weather['Fuel_price'].fillna(method='ffill', inplace=True)

In [83]:
data_weather

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Rank_2018,State,2018,2017,2016,2015,city,lat,lng,population,density,ranking,Fuel_price
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,"3,088"
1,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,NaN,185,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,"3,038"
2,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,"3,038"
3,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,"3,038"
4,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,"3,038"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11035,2013-03-01,LGA,7,5,3,-1,-3,-4,76,63,49,1008,1005,1002,16,16,16,34,21,42.0,0.00,6,NaN,320,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,"3,016"
11036,2013-03-02,LGA,4,2,0,-2,-5,-6,82,65,48,1008,1007,1006,16,15,6,34,20,40.0,T,8,Snow,317,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,"3,016"
11037,2013-03-03,LGA,4,2,-1,-5,-8,-9,69,55,40,1008,1006,1004,16,15,8,39,24,50.0,T,6,Snow,314,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,"3,016"
11038,2013-03-04,LGA,5,2,-2,-7,-8,-9,63,54,44,1012,1009,1008,16,16,16,47,31,60.0,0.00,3,NaN,313,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,"3,005"


In [84]:
data_weather[data_weather['Fuel_price'].isna()]

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Rank_2018,State,2018,2017,2016,2015,city,lat,lng,population,density,ranking,Fuel_price


In [85]:
data_weather['Fuel_price']=data_weather['Fuel_price'].str.replace(',','').astype(int)

In [86]:
data_weather

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Rank_2018,State,2018,2017,2016,2015,city,lat,lng,population,density,ranking,Fuel_price
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,3088
1,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,NaN,185,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,3038
2,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,3038
3,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,3038
4,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,3038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11035,2013-03-01,LGA,7,5,3,-1,-3,-4,76,63,49,1008,1005,1002,16,16,16,34,21,42.0,0.00,6,NaN,320,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,3016
11036,2013-03-02,LGA,4,2,0,-2,-5,-6,82,65,48,1008,1007,1006,16,15,6,34,20,40.0,T,8,Snow,317,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,3016
11037,2013-03-03,LGA,4,2,-1,-5,-8,-9,69,55,40,1008,1006,1004,16,15,8,39,24,50.0,T,6,Snow,314,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,3016
11038,2013-03-04,LGA,5,2,-2,-7,-8,-9,63,54,44,1012,1009,1008,16,16,16,47,31,60.0,0.00,3,NaN,313,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,3005


In [5]:
#data_weather.to_csv('submissions/starting_kit/external_data.csv')

In [6]:
###

NameError: name 'data_weather' is not defined

In [ ]:
holiday = pd.read_csv('data/Jours_fériés_US.csv', sep=';', header=None)
holiday[0]=pd.to_datetime(holiday[0])
holiday

data_weather['Date']=pd.to_datetime(data_weather['Date'])


In [ ]:
data_weather = data_weather.merge(holiday, left_on='Date', right_on=0)
data_weather

In [ ]:
data_weather=data_weather.drop(0, axis=1)

In [ ]:
data_weather=data_weather.rename(
        columns={1: 'Holiday'})

In [ ]:
data_weather

### now

In [35]:
data_weather = pd.read_csv('submissions/starting_kit/external_data.csv')

In [36]:
data = data_weather.drop('Unnamed: 0', axis=1)


In [37]:
time = pd.read_csv('data/air_times.csv')
time = time.drop('Unnamed: 0', axis=1)


In [38]:
time


,ORIGIN,ORIGIN_CITY_NAME,DEST,DEST_CITY_NAME,AIR_TIME_MEAN
0,04A,"Lik, AK",OTZ,"Kotzebue, AK",192.0
1,04A,"Lik, AK",WTK,"Noatak, AK",46.0
2,05A,"Little Squaw, AK",FAI,"Fairbanks, AK",67.4
3,06A,"Kizhuyak, AK",ADQ,"Kodiak, AK",19.5
4,08A,"Elizabeth Island, AK",KEB,"Nanwalek, AK",8.0
...,...,...,...,...,...
24317,ZXN,"Clover Bay, AK",WMK,"Meyers Chuck, AK",29.0
24318,ZXN,"Clover Bay, AK",ZXM,"Saltery Cove, AK",11.5
24319,ZXO,"El Capitan, AK",WFB,"Ketchikan, AK",47.0
24320,ZXU,"North Kingstown, RI",ACK,"Nantucket, MA",38.0


In [39]:
fares = pd.read_csv('data/av_fares_pass.csv')
fares = fares.drop('Unnamed: 0', axis=1)

In [40]:
load_factor = pd.read_csv('data/load_factor_per_month.csv', sep=';')

In [41]:
#pd.DatetimeIndex(df['birth_date']).year
data['Month'] = pd.DatetimeIndex(data['Date']).month
data['Year'] = pd.DatetimeIndex(data['Date']).year
data['Quarter'] = pd.DatetimeIndex(data['Date']).quarter

In [42]:
data = data.merge(load_factor, left_on=['Month', 'Year'], right_on=['Month', 'Year'])
data

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Rank_2018,State,2018,2017,2016,2015,city,lat,lng,population,density,ranking,Fuel_price,Holiday,Month,Year,Quarter,LOAD_FACTOR
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,3088,0,9,2011,3,81.82
1,2011-09-01,ORD,34,28,22,23,22,19,84,70,56,1015,1013,1011,16,15,11,26,9,37.0,0.00,1,NaN,197,3,IL,39874879,38593028,37589899,36305668,Chicago,41.8373,-87.6862,8675982.0,4612.0,1,3088,0,9,2011,3,81.82
2,2011-09-01,LAX,21,18,15,14,14,13,90,76,61,1012,1011,1009,16,11,4,23,13,NaN,0.00,5,Fog,265,2,CA,42626783,41232432,39636042,36351226,Los Angeles,34.1139,-118.4068,12815475.0,3295.0,1,3088,0,9,2011,3,81.82
3,2011-09-01,DFW,38,33,27,17,14,11,51,35,19,1015,1013,1010,16,16,16,34,14,45.0,0.00,1,NaN,157,4,TX,32800721,31816933,31283579,31589832,Dallas,32.7936,-96.7662,5733259.0,1524.0,1,3088,0,9,2011,3,81.82
4,2011-09-01,DEN,36,28,20,5,3,-2,34,22,9,1016,1007,1004,16,16,16,29,15,39.0,0.00,5,NaN,205,5,CO,31363573,29809097,28267394,26280043,Denver,39.7621,-104.8759,2787266.0,1774.0,1,3088,0,9,2011,3,81.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11035,2013-03-05,MSP,-1,-3,-6,-4,-5,-8,100,85,69,1027,1019,1015,16,6,1,27,15,40.0,10.41,8,Fog-Snow,44,17,MN,18363493,18409704,18123844,17634252,Minneapolis,44.9635,-93.2678,2926757.0,3019.0,1,3021,0,3,2013,1,84.99
11036,2013-03-05,DTW,2,-2,-6,-5,-6,-8,81,69,56,1021,1016,1010,16,16,16,32,16,42.0,0.00,5,NaN,58,19,MI,17437716,17036092,16826287,16255520,Detroit,42.3834,-83.1024,3522206.0,1873.0,1,3021,0,3,2013,1,84.99
11037,2013-03-05,PHL,9,4,-1,-2,-6,-7,70,53,36,1018,1016,1014,16,16,16,26,11,32.0,0.00,3,NaN,39,20,PA,15285948,14271243,14564419,15101318,Philadelphia,40.0077,-75.1339,5637884.0,4547.0,1,3021,0,3,2013,1,84.99
11038,2013-03-05,BOS,6,3,0,1,-3,-7,82,68,53,1018,1014,1007,16,16,14,39,17,52.0,T,7,Rain,359,16,MA,20010262,18759742,17749202,16290362,Boston,42.3188,-71.0846,4637537.0,5472.0,1,3021,0,3,2013,1,84.99


In [105]:
data.to_csv('data/our_external.csv')

In [43]:
#fares engineering
fares

,Year,quarter,Departure,Arrival,miles_distance,daily_passengers,average_fare
0,2011,1,"Albany, NY","Chicago, IL",723,213,211.59
1,2011,1,"Albany, NY","Fort Myers, FL",1204,151,229.19
2,2011,1,"Albany, NY","Las Vegas, NV",2237,220,231.51
3,2011,1,"Albany, NY","Miami, FL (Metropolitan Area)",1225,368,200.10
4,2011,1,"Albany, NY","Orlando, FL",1073,624,188.21
...,...,...,...,...,...,...,...
11998,2013,4,"Syracuse, NY","Washington, DC (Metropolitan Area)",298,238,196.27
11999,2013,4,"Tallahassee, FL","Washington, DC (Metropolitan Area)",746,164,223.10
12000,2013,4,"Tampa, FL (Metropolitan Area)","Washington, DC (Metropolitan Area)",842,2688,162.02
12001,2013,4,"Tucson, AZ","Washington, DC (Metropolitan Area)",1976,360,265.80


In [44]:
# new data frame with split value columns 
new = fares["Departure"].str.split(", ", n = 1, expand = True) 
# making separate first name column from new data frame 
fares["city_Dep"]= new[0] 
  
# making separate last name column from new data frame 
fares["State_Dep"]= new[1] 

In [45]:
new = fares["Arrival"].str.split(", ", n = 1, expand = True) 
# making separate first name column from new data frame 
fares["city_Arr"]= new[0] 
  
# making separate last name column from new data frame 
fares["State_Arr"]= new[1].str.split('(').str[0] 


In [46]:
Dep_data=data.add_suffix('_Dep')
Arr_data=data.add_suffix('_Arr')
Dep_data['city_Dep'].unique()

array(['Atlanta', 'Chicago', 'Los Angeles', 'Dallas', 'Denver',
       'New York', 'San Francisco', 'Charlotte', 'Las Vegas', 'Phoenix',
       'Houston', 'Miami', 'Orlando', 'Newark', 'Seattle', 'Minneapolis',
       'Detroit', 'Philadelphia', 'Boston'], dtype=object)

In [47]:
fares['city_Dep'] = fares['city_Dep'].str.split('/').str[0]
fares['city_Dep'] = fares['city_Dep'].str.split(' City').str[0]
fares['city_Arr'] = fares['city_Arr'].str.split('/').str[0]
fares['State_Dep'] = fares['State_Dep'].str.split('(').str[0]
fares['State_Arr'] = fares['State_Arr'].str.split('(').str[0]

In [93]:
fares

,Year,quarter,Departure,Arrival,miles_distance,daily_passengers,average_fare,city_Dep,State_Dep,city_Arr,State_Arr,trajet
0,2011,1,"Albany, NY","Chicago, IL",723,213,211.59,Albany,NY,Chicago,IL,Albany:Chicago
1,2011,1,"Albany, NY","Fort Myers, FL",1204,151,229.19,Albany,NY,Fort Myers,FL,Albany:Chicago
2,2011,1,"Albany, NY","Las Vegas, NV",2237,220,231.51,Albany,NY,Las Vegas,NV,Albany:Chicago
3,2011,1,"Albany, NY","Miami, FL (Metropolitan Area)",1225,368,200.10,Albany,NY,Miami,FL,Albany:Chicago
4,2011,1,"Albany, NY","Orlando, FL",1073,624,188.21,Albany,NY,Orlando,FL,Albany:Chicago
...,...,...,...,...,...,...,...,...,...,...,...,...
11998,2013,4,"Syracuse, NY","Washington, DC (Metropolitan Area)",298,238,196.27,Syracuse,NY,Washington,DC,Syracuse:Houston
11999,2013,4,"Tallahassee, FL","Washington, DC (Metropolitan Area)",746,164,223.10,Tallahassee,FL,Washington,DC,Tallahassee:Houston
12000,2013,4,"Tampa, FL (Metropolitan Area)","Washington, DC (Metropolitan Area)",842,2688,162.02,Tampa,FL,Washington,DC,Tampa:Houston
12001,2013,4,"Tucson, AZ","Washington, DC (Metropolitan Area)",1976,360,265.80,Tucson,AZ,Washington,DC,Tucson:Houston


In [92]:
fares['city_Dep'].unique()
fares['trajet'] = fares['city_Dep']+':'+fares3['city_Arr']
fares['trajet'].unique()

array(['Albany:Chicago', 'Albuquerque:Chicago', 'Allentown:Chicago',
       'Amarillo:Chicago', 'Aspen:Chicago', 'Atlanta:Chicago',
       'Atlantic:Chicago', 'Austin:Chicago', 'Baton Rouge:Chicago',
       'Bellingham:Chicago', 'Billings:Chicago', 'Birmingham:Chicago',
       'Bloomington:Chicago', 'Boise:Chicago', 'Boston:Chicago',
       'Buffalo:Chicago', 'Burlington:Chicago', 'Cedar Rapids:Chicago',
       'Charleston:Chicago', 'Charlotte:Chicago', 'Chicago:Chicago',
       'Cincinnati:Chicago', 'Cleveland:Chicago',
       'Colorado Springs:Chicago', 'Columbia:Chicago', 'Columbus:Chicago',
       'Corpus Christi:Chicago', 'Dallas:Chicago', 'Dayton:Chicago',
       'Denver:Chicago', 'Des Moines:Chicago', 'Detroit:Chicago',
       'Eagle:Chicago', 'El Paso:Chicago', 'Eugene:Chicago',
       'Fargo:Chicago', 'Fayetteville:Chicago', 'Flint:Chicago',
       'Fort Myers:Chicago', 'Fresno:Chicago', 'Grand Rapids:Chicago',
       'Greensboro:Chicago', 'Greenville:Chicago', 'Harlingen:Chic

In [96]:
fares2 = fares.copy()
fares2 = fares2.merge(Dep_data, left_on=['city_Dep','Year','quarter'], right_on=['city_Dep','Year_Dep', 'Quarter_Dep'])
fares2['trajet'].unique()

array(['Atlanta:Dallas', 'Boston:Dallas', 'Charlotte:Dallas',
       'Chicago:Dallas', 'Dallas:Dallas', 'Denver:Dallas',
       'Detroit:Dallas', 'Houston:Dallas', 'Las Vegas:Dallas',
       'Los Angeles:Dallas', 'Los Angeles:Denver', 'Miami:Denver',
       'Minneapolis:Denver', 'New York:Denver', 'Orlando:Denver',
       'Philadelphia:Denver', 'Phoenix:Denver', 'San Francisco:Denver',
       'Seattle:Denver', 'Atlanta:Denver', 'Boston:Denver',
       'Charlotte:Denver', 'Chicago:Denver', 'Dallas:Denver',
       'Denver:Denver', 'Detroit:Denver', 'Houston:Denver',
       'Las Vegas:Denver', 'Dallas:Detroit', 'Denver:Detroit',
       'Detroit:Detroit', 'Houston:Detroit', 'Las Vegas:Detroit',
       'Los Angeles:Detroit', 'Miami:Detroit', 'Minneapolis:Detroit',
       'New York:Detroit', 'Orlando:Detroit', 'Philadelphia:Detroit',
       'Phoenix:Detroit', 'San Francisco:Detroit', 'Seattle:Detroit',
       'Atlanta:Detroit', 'Boston:Detroit', 'Charlotte:Detroit',
       'Chicago:Detroit']

In [97]:
fares2

,Year,quarter,Departure,Arrival,miles_distance,daily_passengers,average_fare,city_Dep,State_Dep_x,city_Arr,State_Arr,trajet,Date_Dep,AirPort_Dep,Max TemperatureC_Dep,Mean TemperatureC_Dep,Min TemperatureC_Dep,Dew PointC_Dep,MeanDew PointC_Dep,Min DewpointC_Dep,Max Humidity_Dep,Mean Humidity_Dep,Min Humidity_Dep,Max Sea Level PressurehPa_Dep,Mean Sea Level PressurehPa_Dep,Min Sea Level PressurehPa_Dep,Max VisibilityKm_Dep,Mean VisibilityKm_Dep,Min VisibilitykM_Dep,Max Wind SpeedKm/h_Dep,Mean Wind SpeedKm/h_Dep,Max Gust SpeedKm/h_Dep,Precipitationmm_Dep,CloudCover_Dep,Events_Dep,WindDirDegrees_Dep,Rank_2018_Dep,State_Dep_y,2018_Dep,2017_Dep,2016_Dep,2015_Dep,lat_Dep,lng_Dep,population_Dep,density_Dep,ranking_Dep,Fuel_price_Dep,Holiday_Dep,Month_Dep,Year_Dep,Quarter_Dep,LOAD_FACTOR_Dep
0,2011,3,"Atlanta, GA (Metropolitan Area)","Atlantic City, NJ",679,200,133.38,Atlanta,GA,Atlantic City,NJ,Atlanta:Dallas,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,1,GA,51866464,50251964,50501858,49340732,33.7627,-84.4225,5228750.0,1384.0,1,3088,0,9,2011,3,81.82
1,2011,3,"Atlanta, GA (Metropolitan Area)","Atlantic City, NJ",679,200,133.38,Atlanta,GA,Atlantic City,NJ,Atlanta:Dallas,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,NaN,185,1,GA,51866464,50251964,50501858,49340732,33.7627,-84.4225,5228750.0,1384.0,1,3038,0,9,2011,3,81.82
2,2011,3,"Atlanta, GA (Metropolitan Area)","Atlantic City, NJ",679,200,133.38,Atlanta,GA,Atlantic City,NJ,Atlanta:Dallas,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147,1,GA,51866464,50251964,50501858,49340732,33.7627,-84.4225,5228750.0,1384.0,1,3038,0,9,2011,3,81.82
3,2011,3,"Atlanta, GA (Metropolitan Area)","Atlantic City, NJ",679,200,133.38,Atlanta,GA,Atlantic City,NJ,Atlanta:Dallas,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139,1,GA,51866464,50251964,50501858,49340732,33.7627,-84.4225,5228750.0,1384.0,1,3038,0,9,2011,3,81.82
4,2011,3,"Atlanta, GA (Metropolitan Area)","Atlantic City, NJ",679,200,133.38,Atlanta,GA,Atlantic City,NJ,Atlanta:Dallas,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149,1,GA,51866464,50251964,50501858,49340732,33.7627,-84.4225,5228750.0,1384.0,1,3038,1,9,2011,3,81.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317998,2013,1,"Seattle, WA","Washington, DC (Metropolitan Area)",2335,1151,308.52,Seattle,WA,Washington,DC,Seattle:Detroit,2013-03-01,SEA,15,13,11,10,9,8,89,78,67,1029,1028,1026,16,15,3,32,19,47.0,4.06,8,Rain,200,8,WA,24894338,22639124,21887110,21231781,47.6211,-122.3244,3643765.0,3336.0,1,3016,0,3,2013,1,84.99
317999,2013,1,"Seattle, WA","Washington, DC (Metropolitan Area)",2335,1151,308.52,Seattle,WA,Washington,DC,Seattle:Detroit,2013-03-02,SEA,14,9,5,8,6,1,83,75,66,1026,1022,1019,16,16,16,29,16,39.0,0.76,8,Rain,199,8,WA,24894338,22639124,21887110,21231781,47.6211,-122.3244,3643765.0,3336.0,1,3016,0,3,2013,1,84.99
318000,2013,1,"Seattle, WA","Washington, DC (Metropolitan Area)",2335,1151,308.52,Seattle,WA,Washington,DC,Seattle:Detroit,2013-03-03,SEA,11,7,2,2,0,-2,89,66,42,1027,1026,1023,16,16,16,21,10,26.0,0.00,3,NaN,134,8,WA,24894338,22639124,21887110,21231781,47.6211,-122.3244,3643765.0,3336.0,1,3016,0,3,2013,1,84.99
318001,2013,1,"Seattle, WA","Washington, DC (Metropolitan Area)",2335,1151,308.52,Seattle,WA,Washington,DC,Seattle:Detroit,2013-03-04,SEA,13,7,0,1,-3,-7,89,58,26,1027,1021,1014,16,16,16,26,14,32.0,0.00,4,NaN,46,8,WA,24894338,22639124,21887110,21231781,47.6211,-122.3244,3643765.0,3336.0,1,3005,0,3,2013,1,84.99


In [100]:
fares3 = fares2.merge(Arr_data, left_on=['city_Arr', 'quarter', 'Year','Date_Dep'], right_on=['city_Arr','Quarter_Arr', 'Year_Arr','Date_Arr'])


In [101]:
fares3

,Year,quarter,Departure,Arrival,miles_distance,daily_passengers,average_fare,city_Dep,State_Dep_x,city_Arr,State_Arr_x,trajet,Date_Dep,AirPort_Dep,Max TemperatureC_Dep,Mean TemperatureC_Dep,Min TemperatureC_Dep,Dew PointC_Dep,MeanDew PointC_Dep,Min DewpointC_Dep,Max Humidity_Dep,Mean Humidity_Dep,Min Humidity_Dep,Max Sea Level PressurehPa_Dep,Mean Sea Level PressurehPa_Dep,Min Sea Level PressurehPa_Dep,Max VisibilityKm_Dep,Mean VisibilityKm_Dep,Min VisibilitykM_Dep,Max Wind SpeedKm/h_Dep,Mean Wind SpeedKm/h_Dep,Max Gust SpeedKm/h_Dep,Precipitationmm_Dep,CloudCover_Dep,Events_Dep,WindDirDegrees_Dep,Rank_2018_Dep,State_Dep_y,2018_Dep,2017_Dep,2016_Dep,2015_Dep,lat_Dep,lng_Dep,population_Dep,density_Dep,ranking_Dep,Fuel_price_Dep,Holiday_Dep,Month_Dep,Year_Dep,Quarter_Dep,LOAD_FACTOR_Dep,Date_Arr,AirPort_Arr,Max TemperatureC_Arr,Mean TemperatureC_Arr,Min TemperatureC_Arr,Dew PointC_Arr,MeanDew PointC_Arr,Min DewpointC_Arr,Max Humidity_Arr,Mean Humidity_Arr,Min Humidity_Arr,Max Sea Level PressurehPa_Arr,Mean Sea Level PressurehPa_Arr,Min Sea Level PressurehPa_Arr,Max VisibilityKm_Arr,Mean VisibilityKm_Arr,Min VisibilitykM_Arr,Max Wind SpeedKm/h_Arr,Mean Wind SpeedKm/h_Arr,Max Gust SpeedKm/h_Arr,Precipitationmm_Arr,CloudCover_Arr,Events_Arr,WindDirDegrees_Arr,Rank_2018_Arr,State_Arr_y,2018_Arr,2017_Arr,2016_Arr,2015_Arr,lat_Arr,lng_Arr,population_Arr,density_Arr,ranking_Arr,Fuel_price_Arr,Holiday_Arr,Month_Arr,Year_Arr,Quarter_Arr,LOAD_FACTOR_Arr
0,2011,3,"Atlanta, GA (Metropolitan Area)","Boston, MA (Metropolitan Area)",952,2335,208.31,Atlanta,GA,Boston,MA,Atlanta:Dallas,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,1,GA,51866464,50251964,50501858,49340732,33.7627,-84.4225,5228750.0,1384.0,1,3088,0,9,2011,3,81.82,2011-09-01,BOS,22,19,17,16,14,12,90,72,53,1024,1023,1021,16,16,14,23,11,26.0,0.00,4,NaN,67,16,MA,20010262,18759742,17749202,16290362,42.3188,-71.0846,4637537.0,5472.0,1,3088,0,9,2011,3,81.82
1,2011,3,"Atlanta, GA (Metropolitan Area)","Boston, MA (Metropolitan Area)",952,2335,208.31,Atlanta,GA,Boston,MA,Atlanta:Dallas,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,NaN,185,1,GA,51866464,50251964,50501858,49340732,33.7627,-84.4225,5228750.0,1384.0,1,3038,0,9,2011,3,81.82,2011-09-02,BOS,20,18,16,15,14,13,90,82,73,1022,1020,1017,16,16,14,21,11,24.0,0.00,3,NaN,84,16,MA,20010262,18759742,17749202,16290362,42.3188,-71.0846,4637537.0,5472.0,1,3038,0,9,2011,3,81.82
2,2011,3,"Atlanta, GA (Metropolitan Area)","Boston, MA (Metropolitan Area)",952,2335,208.31,Atlanta,GA,Boston,MA,Atlanta:Dallas,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147,1,GA,51866464,50251964,50501858,49340732,33.7627,-84.4225,5228750.0,1384.0,1,3038,0,9,2011,3,81.82,2011-09-03,BOS,30,23,16,20,17,13,90,73,55,1017,1013,1011,16,16,10,27,16,37.0,0.00,3,NaN,219,16,MA,20010262,18759742,17749202,16290362,42.3188,-71.0846,4637537.0,5472.0,1,3038,0,9,2011,3,81.82
3,2011,3,"Atlanta, GA (Metropolitan Area)","Boston, MA (Metropolitan Area)",952,2335,208.31,Atlanta,GA,Boston,MA,Atlanta:Dallas,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139,1,GA,51866464,50251964,50501858,49340732,33.7627,-84.4225,5228750.0,1384.0,1,3038,0,9,2011,3,81.82,2011-09-04,BOS,29,25,20,22,19,17,84,73,61,1013,1011,1009,16,14,8,34,19,40.0,0.00,5,NaN,209,16,MA,20010262,18759742,17749202,16290362,42.3188,-71.0846,4637537.0,5472.0,1,3038,0,9,2011,3,81.82
4,2011,3,"Atlanta, GA (Metropolitan Area)","Boston, MA (Metropolitan Area)",952,2335,208.31,Atlanta,GA,Boston,MA,Atlanta:Dallas,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149,1,GA,51866464,50251964,50501858,49340732,33.7627,-84.4225,5228750.0,1384.0,1,3038,1,9,2011,3,81.82,2011-09-05,BOS,31,26,21,21,19,16,87,69,51,1010,1010,1009,16,16,14,37,22,48.0,0.00,5,NaN,204,16,MA,20010262,18759742,17749202,16290362,42.3188,-71.0846,4637537.0,5472

In [103]:
fares3.to_csv('data/big_external.csv')

In [102]:
fares3['trajet'].unique()

array(['Atlanta:Dallas', 'Boston:Dallas', 'Charlotte:Dallas',
       'Chicago:Dallas', 'Dallas:Dallas', 'Denver:Dallas',
       'Detroit:Dallas', 'Houston:Dallas', 'Las Vegas:Dallas',
       'Los Angeles:Dallas', 'Los Angeles:Denver', 'Miami:Denver',
       'Minneapolis:Denver', 'New York:Denver', 'Orlando:Denver',
       'Philadelphia:Denver', 'Phoenix:Denver', 'San Francisco:Denver',
       'Atlanta:Denver', 'Boston:Denver', 'Charlotte:Denver',
       'Chicago:Denver', 'Dallas:Denver', 'Denver:Denver',
       'Detroit:Denver', 'Houston:Denver', 'Las Vegas:Denver',
       'Denver:Detroit', 'Detroit:Detroit', 'Houston:Detroit',
       'Las Vegas:Detroit', 'Los Angeles:Detroit', 'Miami:Detroit',
       'Minneapolis:Detroit', 'New York:Detroit', 'Orlando:Detroit',
       'Philadelphia:Detroit', 'Dallas:Detroit', 'Phoenix:Detroit',
       'San Francisco:Detroit', 'Atlanta:Detroit', 'Boston:Detroit',
       'Charlotte:Detroit', 'Chicago:Detroit'], dtype=object)

In [106]:
fares3

,Year,quarter,Departure,Arrival,miles_distance,daily_passengers,average_fare,city_Dep,State_Dep_x,city_Arr,State_Arr_x,trajet,Date_Dep,AirPort_Dep,Max TemperatureC_Dep,Mean TemperatureC_Dep,Min TemperatureC_Dep,Dew PointC_Dep,MeanDew PointC_Dep,Min DewpointC_Dep,Max Humidity_Dep,Mean Humidity_Dep,Min Humidity_Dep,Max Sea Level PressurehPa_Dep,Mean Sea Level PressurehPa_Dep,Min Sea Level PressurehPa_Dep,Max VisibilityKm_Dep,Mean VisibilityKm_Dep,Min VisibilitykM_Dep,Max Wind SpeedKm/h_Dep,Mean Wind SpeedKm/h_Dep,Max Gust SpeedKm/h_Dep,Precipitationmm_Dep,CloudCover_Dep,Events_Dep,WindDirDegrees_Dep,Rank_2018_Dep,State_Dep_y,2018_Dep,2017_Dep,2016_Dep,2015_Dep,lat_Dep,lng_Dep,population_Dep,density_Dep,ranking_Dep,Fuel_price_Dep,Holiday_Dep,Month_Dep,Year_Dep,Quarter_Dep,LOAD_FACTOR_Dep,Date_Arr,AirPort_Arr,Max TemperatureC_Arr,Mean TemperatureC_Arr,Min TemperatureC_Arr,Dew PointC_Arr,MeanDew PointC_Arr,Min DewpointC_Arr,Max Humidity_Arr,Mean Humidity_Arr,Min Humidity_Arr,Max Sea Level PressurehPa_Arr,Mean Sea Level PressurehPa_Arr,Min Sea Level PressurehPa_Arr,Max VisibilityKm_Arr,Mean VisibilityKm_Arr,Min VisibilitykM_Arr,Max Wind SpeedKm/h_Arr,Mean Wind SpeedKm/h_Arr,Max Gust SpeedKm/h_Arr,Precipitationmm_Arr,CloudCover_Arr,Events_Arr,WindDirDegrees_Arr,Rank_2018_Arr,State_Arr_y,2018_Arr,2017_Arr,2016_Arr,2015_Arr,lat_Arr,lng_Arr,population_Arr,density_Arr,ranking_Arr,Fuel_price_Arr,Holiday_Arr,Month_Arr,Year_Arr,Quarter_Arr,LOAD_FACTOR_Arr
0,2011,3,"Atlanta, GA (Metropolitan Area)","Boston, MA (Metropolitan Area)",952,2335,208.31,Atlanta,GA,Boston,MA,Atlanta:Dallas,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,1,GA,51866464,50251964,50501858,49340732,33.7627,-84.4225,5228750.0,1384.0,1,3088,0,9,2011,3,81.82,2011-09-01,BOS,22,19,17,16,14,12,90,72,53,1024,1023,1021,16,16,14,23,11,26.0,0.00,4,NaN,67,16,MA,20010262,18759742,17749202,16290362,42.3188,-71.0846,4637537.0,5472.0,1,3088,0,9,2011,3,81.82
1,2011,3,"Atlanta, GA (Metropolitan Area)","Boston, MA (Metropolitan Area)",952,2335,208.31,Atlanta,GA,Boston,MA,Atlanta:Dallas,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,NaN,185,1,GA,51866464,50251964,50501858,49340732,33.7627,-84.4225,5228750.0,1384.0,1,3038,0,9,2011,3,81.82,2011-09-02,BOS,20,18,16,15,14,13,90,82,73,1022,1020,1017,16,16,14,21,11,24.0,0.00,3,NaN,84,16,MA,20010262,18759742,17749202,16290362,42.3188,-71.0846,4637537.0,5472.0,1,3038,0,9,2011,3,81.82
2,2011,3,"Atlanta, GA (Metropolitan Area)","Boston, MA (Metropolitan Area)",952,2335,208.31,Atlanta,GA,Boston,MA,Atlanta:Dallas,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147,1,GA,51866464,50251964,50501858,49340732,33.7627,-84.4225,5228750.0,1384.0,1,3038,0,9,2011,3,81.82,2011-09-03,BOS,30,23,16,20,17,13,90,73,55,1017,1013,1011,16,16,10,27,16,37.0,0.00,3,NaN,219,16,MA,20010262,18759742,17749202,16290362,42.3188,-71.0846,4637537.0,5472.0,1,3038,0,9,2011,3,81.82
3,2011,3,"Atlanta, GA (Metropolitan Area)","Boston, MA (Metropolitan Area)",952,2335,208.31,Atlanta,GA,Boston,MA,Atlanta:Dallas,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139,1,GA,51866464,50251964,50501858,49340732,33.7627,-84.4225,5228750.0,1384.0,1,3038,0,9,2011,3,81.82,2011-09-04,BOS,29,25,20,22,19,17,84,73,61,1013,1011,1009,16,14,8,34,19,40.0,0.00,5,NaN,209,16,MA,20010262,18759742,17749202,16290362,42.3188,-71.0846,4637537.0,5472.0,1,3038,0,9,2011,3,81.82
4,2011,3,"Atlanta, GA (Metropolitan Area)","Boston, MA (Metropolitan Area)",952,2335,208.31,Atlanta,GA,Boston,MA,Atlanta:Dallas,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149,1,GA,51866464,50251964,50501858,49340732,33.7627,-84.4225,5228750.0,1384.0,1,3038,1,9,2011,3,81.82,2011-09-05,BOS,31,26,21,21,19,16,87,69,51,1010,1010,1009,16,16,14,37,22,48.0,0.00,5,NaN,204,16,MA,20010262,18759742,17749202,16290362,42.3188,-71.0846,4637537.0,5472

In [109]:
faresco = pd.read_csv('data/faresco.csv')
faresco = faresco.drop('Unnamed: 0', axis=1)

In [110]:
faresco

,Year,quarter,Departure,Arrival,miles_distance,daily_passengers,average_fare
0,2011,1,"Atlanta, GA (Metropolitan Area)","Boston, MA (Metropolitan Area)",952,1671,215.75
1,2011,2,"Atlanta, GA (Metropolitan Area)","Boston, MA (Metropolitan Area)",952,2295,218.02
2,2011,3,"Atlanta, GA (Metropolitan Area)","Boston, MA (Metropolitan Area)",952,2335,208.31
3,2011,4,"Atlanta, GA (Metropolitan Area)","Boston, MA (Metropolitan Area)",952,2117,227.89
4,2012,1,"Atlanta, GA (Metropolitan Area)","Boston, MA (Metropolitan Area)",952,1704,235.04
...,...,...,...,...,...,...,...
1819,2012,4,"San Francisco, CA (Metropolitan Area)","Seattle, WA",697,4952,169.63
1820,2013,1,"San Francisco, CA (Metropolitan Area)","Seattle, WA",697,4434,162.59
1821,2013,2,"San Francisco, CA (Metropolitan Area)","Seattle, WA",697,5382,165.23
1822,2013,3,"San Francisco, CA (Metropolitan Area)","Seattle, WA",697,5934,162.96


In [112]:
# new data frame with split value columns 
new = faresco["Departure"].str.split(", ", n = 1, expand = True) 
# making separate first name column from new data frame 
faresco["city_Dep"]= new[0] 
new = faresco["Arrival"].str.split(", ", n = 1, expand = True) 
# making separate first name column from new data frame 
faresco["city_Arr"]= new[0] 
  
# making separate last name column from new data frame 
faresco["State_Arr"]= new[1].str.split('(').str[0] 

# making separate last name column from new data frame 
faresco["State_Dep"]= new[1] 
faresco['city_Dep'] = faresco['city_Dep'].str.split('/').str[0]
faresco['city_Dep'] = faresco['city_Dep'].str.split(' City').str[0]
faresco['city_Arr'] = faresco['city_Arr'].str.split('/').str[0]
faresco['State_Dep'] = faresco['State_Dep'].str.split('(').str[0]
faresco['State_Arr'] = faresco['State_Arr'].str.split('(').str[0]

In [113]:
faresco

,Year,quarter,Departure,Arrival,miles_distance,daily_passengers,average_fare,city_Dep,city_Arr,State_Arr,State_Dep
0,2011,1,"Atlanta, GA (Metropolitan Area)","Boston, MA (Metropolitan Area)",952,1671,215.75,Atlanta,Boston,MA,MA
1,2011,2,"Atlanta, GA (Metropolitan Area)","Boston, MA (Metropolitan Area)",952,2295,218.02,Atlanta,Boston,MA,MA
2,2011,3,"Atlanta, GA (Metropolitan Area)","Boston, MA (Metropolitan Area)",952,2335,208.31,Atlanta,Boston,MA,MA
3,2011,4,"Atlanta, GA (Metropolitan Area)","Boston, MA (Metropolitan Area)",952,2117,227.89,Atlanta,Boston,MA,MA
4,2012,1,"Atlanta, GA (Metropolitan Area)","Boston, MA (Metropolitan Area)",952,1704,235.04,Atlanta,Boston,MA,MA
...,...,...,...,...,...,...,...,...,...,...,...
1819,2012,4,"San Francisco, CA (Metropolitan Area)","Seattle, WA",697,4952,169.63,San Francisco,Seattle,WA,WA
1820,2013,1,"San Francisco, CA (Metropolitan Area)","Seattle, WA",697,4434,162.59,San Francisco,Seattle,WA,WA
1821,2013,2,"San Francisco, CA (Metropolitan Area)","Seattle, WA",697,5382,165.23,San Francisco,Seattle,WA,WA
1822,2013,3,"San Francisco, CA (Metropolitan Area)","Seattle, WA",697,5934,162.96,San Francisco,Seattle,WA,WA


In [116]:
faresco = faresco.drop('Arrival', axis=1)

In [117]:
faresco

,Year,quarter,miles_distance,daily_passengers,average_fare,city_Dep,city_Arr,State_Arr,State_Dep
0,2011,1,952,1671,215.75,Atlanta,Boston,MA,MA
1,2011,2,952,2295,218.02,Atlanta,Boston,MA,MA
2,2011,3,952,2335,208.31,Atlanta,Boston,MA,MA
3,2011,4,952,2117,227.89,Atlanta,Boston,MA,MA
4,2012,1,952,1704,235.04,Atlanta,Boston,MA,MA
...,...,...,...,...,...,...,...,...,...
1819,2012,4,697,4952,169.63,San Francisco,Seattle,WA,WA
1820,2013,1,697,4434,162.59,San Francisco,Seattle,WA,WA
1821,2013,2,697,5382,165.23,San Francisco,Seattle,WA,WA
1822,2013,3,697,5934,162.96,San Francisco,Seattle,WA,WA


In [154]:
iata['Major city served'] = iata['Major city served'].str.replace('\xa0', '')

In [160]:
f = faresco.merge(iata[['IATA Code', 'Major city served']], left_on='city_Dep', right_on='Major city served')
f = f.drop('Major city served', axis=1)

In [163]:
f = f.merge(iata[['IATA Code', 'Major city served']], left_on='city_Arr', right_on='Major city served')
f = f.drop('Major city served', axis=1)

In [164]:
f

,Year,quarter,miles_distance,daily_passengers,average_fare,city_Dep,city_Arr,State_Arr,State_Dep,IATA Code_x,IATA Code_y
0,2011,1,952,1671,215.75,Atlanta,Boston,MA,MA,ATL,BOS
1,2011,2,952,2295,218.02,Atlanta,Boston,MA,MA,ATL,BOS
2,2011,3,952,2335,208.31,Atlanta,Boston,MA,MA,ATL,BOS
3,2011,4,952,2117,227.89,Atlanta,Boston,MA,MA,ATL,BOS
4,2012,1,952,1704,235.04,Atlanta,Boston,MA,MA,ATL,BOS
...,...,...,...,...,...,...,...,...,...,...,...
1915,2012,4,697,4952,169.63,San Francisco,Seattle,WA,WA,SFO,SEA
1916,2013,1,697,4434,162.59,San Francisco,Seattle,WA,WA,SFO,SEA
1917,2013,2,697,5382,165.23,San Francisco,Seattle,WA,WA,SFO,SEA
1918,2013,3,697,5934,162.96,San Francisco,Seattle,WA,WA,SFO,SEA


In [168]:
f = f.rename(
        columns={'IATA Code_x': 'IATA_Dep', 'IATA Code_y': 'IATA_Arr'})

In [169]:
f

,Year,quarter,miles_distance,daily_passengers,average_fare,city_Dep,city_Arr,State_Arr,State_Dep,IATA_Dep,IATA_Arr
0,2011,1,952,1671,215.75,Atlanta,Boston,MA,MA,ATL,BOS
1,2011,2,952,2295,218.02,Atlanta,Boston,MA,MA,ATL,BOS
2,2011,3,952,2335,208.31,Atlanta,Boston,MA,MA,ATL,BOS
3,2011,4,952,2117,227.89,Atlanta,Boston,MA,MA,ATL,BOS
4,2012,1,952,1704,235.04,Atlanta,Boston,MA,MA,ATL,BOS
...,...,...,...,...,...,...,...,...,...,...,...
1915,2012,4,697,4952,169.63,San Francisco,Seattle,WA,WA,SFO,SEA
1916,2013,1,697,4434,162.59,San Francisco,Seattle,WA,WA,SFO,SEA
1917,2013,2,697,5382,165.23,San Francisco,Seattle,WA,WA,SFO,SEA
1918,2013,3,697,5934,162.96,San Francisco,Seattle,WA,WA,SFO,SEA


In [172]:
ext = pd.read_csv('data/our_external.csv')
ext = ext.drop('Unnamed: 0', axis=1)

In [176]:
ext_dep =ext.add_suffix('_Dep')
ext_arr =ext.add_suffix('_Arr')

In [184]:
ext_arr

,Date_Arr,AirPort_Arr,Max TemperatureC_Arr,Mean TemperatureC_Arr,Min TemperatureC_Arr,Dew PointC_Arr,MeanDew PointC_Arr,Min DewpointC_Arr,Max Humidity_Arr,Mean Humidity_Arr,Min Humidity_Arr,Max Sea Level PressurehPa_Arr,Mean Sea Level PressurehPa_Arr,Min Sea Level PressurehPa_Arr,Max VisibilityKm_Arr,Mean VisibilityKm_Arr,Min VisibilitykM_Arr,Max Wind SpeedKm/h_Arr,Mean Wind SpeedKm/h_Arr,Max Gust SpeedKm/h_Arr,Precipitationmm_Arr,CloudCover_Arr,Events_Arr,WindDirDegrees_Arr,Rank_2018_Arr,State_Arr,2018_Arr,2017_Arr,2016_Arr,2015_Arr,city_Arr,lat_Arr,lng_Arr,population_Arr,density_Arr,ranking_Arr,Fuel_price_Arr,Holiday_Arr,Month_Arr,Year_Arr,Quarter_Arr,LOAD_FACTOR_Arr
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,3088,0,9,2011,3,81.82
1,2011-09-01,ORD,34,28,22,23,22,19,84,70,56,1015,1013,1011,16,15,11,26,9,37.0,0.00,1,NaN,197,3,IL,39874879,38593028,37589899,36305668,Chicago,41.8373,-87.6862,8675982.0,4612.0,1,3088,0,9,2011,3,81.82
2,2011-09-01,LAX,21,18,15,14,14,13,90,76,61,1012,1011,1009,16,11,4,23,13,NaN,0.00,5,Fog,265,2,CA,42626783,41232432,39636042,36351226,Los Angeles,34.1139,-118.4068,12815475.0,3295.0,1,3088,0,9,2011,3,81.82
3,2011-09-01,DFW,38,33,27,17,14,11,51,35,19,1015,1013,1010,16,16,16,34,14,45.0,0.00,1,NaN,157,4,TX,32800721,31816933,31283579,31589832,Dallas,32.7936,-96.7662,5733259.0,1524.0,1,3088,0,9,2011,3,81.82
4,2011-09-01,DEN,36,28,20,5,3,-2,34,22,9,1016,1007,1004,16,16,16,29,15,39.0,0.00,5,NaN,205,5,CO,31363573,29809097,28267394,26280043,Denver,39.7621,-104.8759,2787266.0,1774.0,1,3088,0,9,2011,3,81.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11035,2013-03-05,MSP,-1,-3,-6,-4,-5,-8,100,85,69,1027,1019,1015,16,6,1,27,15,40.0,10.41,8,Fog-Snow,44,17,MN,18363493,18409704,18123844,17634252,Minneapolis,44.9635,-93.2678,2926757.0,3019.0,1,3021,0,3,2013,1,84.99
11036,2013-03-05,DTW,2,-2,-6,-5,-6,-8,81,69,56,1021,1016,1010,16,16,16,32,16,42.0,0.00,5,NaN,58,19,MI,17437716,17036092,16826287,16255520,Detroit,42.3834,-83.1024,3522206.0,1873.0,1,3021,0,3,2013,1,84.99
11037,2013-03-05,PHL,9,4,-1,-2,-6,-7,70,53,36,1018,1016,1014,16,16,16,26,11,32.0,0.00,3,NaN,39,20,PA,15285948,14271243,14564419,15101318,Philadelphia,40.0077,-75.1339,5637884.0,4547.0,1,3021,0,3,2013,1,84.99
11038,2013-03-05,BOS,6,3,0,1,-3,-7,82,68,53,1018,1014,1007,16,16,14,39,17,52.0,T,7,Rain,359,16,MA,20010262,18759742,17749202,16290362,Boston,42.3188,-71.0846,4637537.0,5472.0,1,3021,0,3,2013,1,84.99


In [178]:
final = f.merge(ext_dep, left_on=['IATA_Dep','Year', 'quarter'], right_on=['AirPort_Dep', 'Year_Dep', 'Quarter_Dep'])

In [182]:
final =final.rename(columns={'city_Dep_x':'city_Dep', 'State_Dep_x':'State_Dep'})

In [183]:
final

,Year,quarter,miles_distance,daily_passengers,average_fare,city_Dep,city_Arr,State_Arr,State_Dep,IATA_Dep,IATA_Arr,Date_Dep,AirPort_Dep,Max TemperatureC_Dep,Mean TemperatureC_Dep,Min TemperatureC_Dep,Dew PointC_Dep,MeanDew PointC_Dep,Min DewpointC_Dep,Max Humidity_Dep,Mean Humidity_Dep,Min Humidity_Dep,Max Sea Level PressurehPa_Dep,Mean Sea Level PressurehPa_Dep,Min Sea Level PressurehPa_Dep,Max VisibilityKm_Dep,Mean VisibilityKm_Dep,Min VisibilitykM_Dep,Max Wind SpeedKm/h_Dep,Mean Wind SpeedKm/h_Dep,Max Gust SpeedKm/h_Dep,Precipitationmm_Dep,CloudCover_Dep,Events_Dep,WindDirDegrees_Dep,Rank_2018_Dep,2018_Dep,2017_Dep,2016_Dep,2015_Dep,lat_Dep,lng_Dep,population_Dep,density_Dep,ranking_Dep,Fuel_price_Dep,Holiday_Dep,LOAD_FACTOR_Dep
0,2011,3,952,2335,208.31,Atlanta,Boston,MA,MA,ATL,BOS,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,1,51866464,50251964,50501858,49340732,33.7627,-84.4225,5228750.0,1384.0,1,3088,0,81.82
1,2011,3,952,2335,208.31,Atlanta,Boston,MA,MA,ATL,BOS,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,NaN,185,1,51866464,50251964,50501858,49340732,33.7627,-84.4225,5228750.0,1384.0,1,3038,0,81.82
2,2011,3,952,2335,208.31,Atlanta,Boston,MA,MA,ATL,BOS,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147,1,51866464,50251964,50501858,49340732,33.7627,-84.4225,5228750.0,1384.0,1,3038,0,81.82
3,2011,3,952,2335,208.31,Atlanta,Boston,MA,MA,ATL,BOS,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139,1,51866464,50251964,50501858,49340732,33.7627,-84.4225,5228750.0,1384.0,1,3038,0,81.82
4,2011,3,952,2335,208.31,Atlanta,Boston,MA,MA,ATL,BOS,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149,1,51866464,50251964,50501858,49340732,33.7627,-84.4225,5228750.0,1384.0,1,3038,1,81.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81139,2013,1,697,4434,162.59,San Francisco,Seattle,WA,WA,SFO,SEA,2013-03-01,SFO,22,15,8,11,8,6,100,75,49,1031,1028,1024,16,10,0,32,7,37.0,0.00,3,Fog,330,7,27794154,26900048,25707101,24190549,37.7562,-122.4430,3603761.0,7286.0,1,3016,0,84.99
81140,2013,1,697,4434,162.59,San Francisco,Seattle,WA,WA,SFO,SEA,2013-03-02,SFO,17,14,11,11,9,8,89,76,62,1024,1021,1019,16,16,16,27,9,34.0,0.00,6,NaN,307,7,27794154,26900048,25707101,24190549,37.7562,-122.4430,3603761.0,7286.0,1,3016,0,84.99
81141,2013,1,697,4434,162.59,San Francisco,Seattle,WA,WA,SFO,SEA,2013-03-03,SFO,15,12,9,8,7,6,83,73,62,1020,1019,1019,16,15,14,39,20,48.0,T,7,NaN,276,7,27794154,26900048,25707101,24190549,37.7562,-122.4430,3603761.0,7286.0,1,3016,0,84.99
81142,2013,1,697,4434,162.59,San Francisco,Seattle,WA,WA,SFO,SEA,2013-03-04,SFO,13,11,8,6,5,4,86,73,59,1022,1020,1019,16,16,14,39,15,47.0,0.00,5,NaN,267,7,27794154,26900048,25707101,24190549,37.7562,-122.4430,3603761.0,7286.0,1,3005,0,84.99


In [185]:
final_ext = final.merge(ext_arr, left_on=['IATA_Arr', 'Date_Dep'], right_on=['AirPort_Arr', 'Date_Arr'])

In [186]:
final_ext

,Year,quarter,miles_distance,daily_passengers,average_fare,city_Dep,city_Arr_x,State_Arr_x,State_Dep,IATA_Dep,IATA_Arr,Date_Dep,AirPort_Dep,Max TemperatureC_Dep,Mean TemperatureC_Dep,Min TemperatureC_Dep,Dew PointC_Dep,MeanDew PointC_Dep,Min DewpointC_Dep,Max Humidity_Dep,Mean Humidity_Dep,Min Humidity_Dep,Max Sea Level PressurehPa_Dep,Mean Sea Level PressurehPa_Dep,Min Sea Level PressurehPa_Dep,Max VisibilityKm_Dep,Mean VisibilityKm_Dep,Min VisibilitykM_Dep,Max Wind SpeedKm/h_Dep,Mean Wind SpeedKm/h_Dep,Max Gust SpeedKm/h_Dep,Precipitationmm_Dep,CloudCover_Dep,Events_Dep,WindDirDegrees_Dep,Rank_2018_Dep,2018_Dep,2017_Dep,2016_Dep,2015_Dep,lat_Dep,lng_Dep,population_Dep,density_Dep,ranking_Dep,Fuel_price_Dep,Holiday_Dep,LOAD_FACTOR_Dep,Date_Arr,AirPort_Arr,Max TemperatureC_Arr,Mean TemperatureC_Arr,Min TemperatureC_Arr,Dew PointC_Arr,MeanDew PointC_Arr,Min DewpointC_Arr,Max Humidity_Arr,Mean Humidity_Arr,Min Humidity_Arr,Max Sea Level PressurehPa_Arr,Mean Sea Level PressurehPa_Arr,Min Sea Level PressurehPa_Arr,Max VisibilityKm_Arr,Mean VisibilityKm_Arr,Min VisibilitykM_Arr,Max Wind SpeedKm/h_Arr,Mean Wind SpeedKm/h_Arr,Max Gust SpeedKm/h_Arr,Precipitationmm_Arr,CloudCover_Arr,Events_Arr,WindDirDegrees_Arr,Rank_2018_Arr,State_Arr_y,2018_Arr,2017_Arr,2016_Arr,2015_Arr,city_Arr_y,lat_Arr,lng_Arr,population_Arr,density_Arr,ranking_Arr,Fuel_price_Arr,Holiday_Arr,Month_Arr,Year_Arr,Quarter_Arr,LOAD_FACTOR_Arr
0,2011,3,952,2335,208.31,Atlanta,Boston,MA,MA,ATL,BOS,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,1,51866464,50251964,50501858,49340732,33.7627,-84.4225,5228750.0,1384.0,1,3088,0,81.82,2011-09-01,BOS,22,19,17,16,14,12,90,72,53,1024,1023,1021,16,16,14,23,11,26.0,0.00,4,NaN,67,16,MA,20010262,18759742,17749202,16290362,Boston,42.3188,-71.0846,4637537.0,5472.0,1,3088,0,9,2011,3,81.82
1,2011,3,952,2335,208.31,Atlanta,Boston,MA,MA,ATL,BOS,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,NaN,185,1,51866464,50251964,50501858,49340732,33.7627,-84.4225,5228750.0,1384.0,1,3038,0,81.82,2011-09-02,BOS,20,18,16,15,14,13,90,82,73,1022,1020,1017,16,16,14,21,11,24.0,0.00,3,NaN,84,16,MA,20010262,18759742,17749202,16290362,Boston,42.3188,-71.0846,4637537.0,5472.0,1,3038,0,9,2011,3,81.82
2,2011,3,952,2335,208.31,Atlanta,Boston,MA,MA,ATL,BOS,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147,1,51866464,50251964,50501858,49340732,33.7627,-84.4225,5228750.0,1384.0,1,3038,0,81.82,2011-09-03,BOS,30,23,16,20,17,13,90,73,55,1017,1013,1011,16,16,10,27,16,37.0,0.00,3,NaN,219,16,MA,20010262,18759742,17749202,16290362,Boston,42.3188,-71.0846,4637537.0,5472.0,1,3038,0,9,2011,3,81.82
3,2011,3,952,2335,208.31,Atlanta,Boston,MA,MA,ATL,BOS,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139,1,51866464,50251964,50501858,49340732,33.7627,-84.4225,5228750.0,1384.0,1,3038,0,81.82,2011-09-04,BOS,29,25,20,22,19,17,84,73,61,1013,1011,1009,16,14,8,34,19,40.0,0.00,5,NaN,209,16,MA,20010262,18759742,17749202,16290362,Boston,42.3188,-71.0846,4637537.0,5472.0,1,3038,0,9,2011,3,81.82
4,2011,3,952,2335,208.31,Atlanta,Boston,MA,MA,ATL,BOS,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149,1,51866464,50251964,50501858,49340732,33.7627,-84.4225,5228750.0,1384.0,1,3038,1,81.82,2011-09-05,BOS,31,26,21,21,19,16,87,69,51,1010,1010,1009,16,16,14,37,22,48.0,0.00,5,NaN,204,16,MA,20010262,18759742,17749202,16290362,Boston,42.3188,-71.0846,4637537.0,5472.0,1,3038,1,9,2011,3,81.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79483,2013,1,2448,1597,280.97,New York,Seat

In [203]:
final_ext.to_csv('data/final_external.csv')